In [7]:
import logging
import random
import time

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as ec
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.ui import WebDriverWait as wait




from selenium import webdriver
from selenium.webdriver.chrome.options import Options

In [48]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from typing import Dict
from orb.utils import GetUserAgent
from orb.utils import GetProxies
from webdriver_manager.chrome import ChromeDriverManager

import logging
import random

from selenium.webdriver.common.by import By

from orb.spinner.utils.spoofing import DriverSpoofing

log = logging.getLogger(__name__)



class OrbDriver:

    def __init__(self, *args, **kwargs):
        self.driver_spoof = DriverSpoofing()

        if kwargs.get('headless'):
            self.headless = kwargs['headless']
        else:
            self.headless = None

    @property
    def random_user_agent(self) -> 'GetUserAgent':
        return GetUserAgent().headers_dict['User-Agent']

    @property
    def random_proxy(self) -> 'GetProxies':
        return GetProxies().proxy_dict['https']

    def _webdriver_options_init(self):

        # Initialise webdriver options
        self.webdriver_options = Options()

        # Default webdriver options
        # Disable javascript
        self.webdriver_options.add_argument("--disable-javascript")

        # Set to headless if applicable
        if self.headless:
            self.webdriver_options.add_argument("--headless")

        # Set the user agent string
        user_agent = self.random_user_agent
        log.info(f"Initialising webdriver with user-agent: {user_agent} ")
        self.webdriver_options.add_argument(f"user-agent={user_agent}")

        # Set the proxy string
        proxy = self.random_proxy
        log.info(f"Initialising webdriver with proxy: {proxy} ")
        self.webdriver_options.add_argument(f'--proxy-server={proxy}')

    def change_driver_viewport(self):

        self.driver_spoof.change_viewport_size(
            driver=self.driver
        )

    def get_webdriver(self, url: str = None):

        # Always install the latest web driver
        driver_install = ChromeDriverManager().install()
        
        # Initialise webdriver options
        self._webdriver_options_init()

        # Initialize the webdriver with the options
        self.driver = webdriver.Chrome(driver_install, options=self.webdriver_options)
        
        if url:
            return self.driver.get(url=url)
        return self.driver


In [49]:
from selenium.webdriver.common.action_chains import ActionChains